In [ ]:
!pip install catboost

In [ ]:
import pandas as pd
import os
import numpy as np
import catboost
from catboost import *
from catboost import datasets
from catboost import CatBoostRegressor
from catboost import cv

from sklearn.model_selection import train_test_split

RAND = 10


In [ ]:
df = pd.read_csv('./train.csv').fillna(0)

In [ ]:
#разделим targen и features 
y = df.target
X = df.drop('target', axis=1)

In [ ]:
df.columns

Index(['flag_woman', 'flag_man', 'flag_resident', 'flag_non_resident', 'city',
       'flag_own_car', 'flag_own_realty', 'flag_phone', 'flag_work_phone',
       'flag_email', 'businessman_flag', 'amount_children', 'annual_incom',
       'salary', 'family_income', 'birth_date', 'family_status',
       'level_education', 'target'],
      dtype='object')

In [137]:
X = X.drop(['city', 'birth_date', 'family_status', 'level_education'], axis=1)

In [118]:
pool = Pool(
    data=X,
    label=y,
    cat_features=cat_features
)
print('Dataset shape:' + str(pool.shape))
print('Column name')
print(pool.get_feature_names())

Dataset shape:(16797, 18)
Column name
['flag_woman', 'flag_man', 'flag_resident', 'flag_non_resident', 'city', 'flag_own_car', 'flag_own_realty', 'flag_phone', 'flag_work_phone', 'flag_email', 'businessman_flag', 'amount_children', 'annual_income', 'salary', 'family_income', 'birth_date', 'family_status', 'level_education']


In [140]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.8, random_state=RAND)


In [ ]:
model = CatBoostRegressor(loss_function='MAE',
    iterations=2000,
    learning_rate=0.1,
    depth=6,
    random_seed=RAND,
    use_best_model=True,
    early_stopping_rounds=20
)  
train_dataset = Pool(
    data=X_train,
    label=y_train,
    cat_features=cat_features
)
    
eval_dataset = Pool(
    data=X_validation,
    label=y_validation,
    cat_features=cat_features
)
    
model.fit(train_dataset,
          use_best_model=True,
          verbose = 1,
          eval_set=eval_dataset,
)

In [88]:
print('Model is fitted: ' + str(model.is_fitted()))
print('Model params:')
print(model.get_params())
print('Tree count: ' + str(model.tree_count_))

Model is fitted: True
Model params:
{'iterations': 2000, 'learning_rate': 0.3, 'depth': 6, 'loss_function': 'MultiRMSE', 'random_seed': 10, 'use_best_model': True}
Tree count: 21


In [ ]:
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.3,
    depth=1,
    random_seed=RAND,
    custom_loss=['AUC', 'Accuracy'],
    use_best_model=True,
    early_stopping_rounds=20,
    #loss_function='CrossEntropy'
)
model.fit(
    pool,
    cat_features=cat_features,
    plot=True
)
print('Model is fitted: ' + str(model.is_fitted()))
print('Model params:')
print(model.get_params())
print('Tree count: ' + str(model.tree_count_))

In [102]:
model.save_model('model.json', format='json')